In [ ]:
%run base.ipynb

In [ ]:
experiment = 'few-shot-seeds-mulneg'
experiment = 'more-few-shot-seeds'

results_path = get_experiment_path(experiment)

In [ ]:
results_path

In [ ]:
model = 'all-mpnet'

In [ ]:
pd.options.display.max_rows = None

In [ ]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

def format_f(x):
    return re.sub('^0\.', '.', f'{x:.4f}')

def read_results(name, templates=[]):
    paths = list(results_path.glob(f'{model}*/results/{name}.json'))
    dfs = []
    for path in tqdm(paths):
#         print(path)
        df = pd.read_json(path, orient='index')
        if len(df.columns) == 1:
            df = pd.DataFrame([df.to_dict()[0]], columns=df.to_dict()[0].keys())
#         print(df)
#         df[['model','template','per_map_size', 'map_size', 'seed']] = re.findall(r'([a-zA-Z]+-[a-zA-Z]+)-([a-zA-Z_]*)-n(\d+)in(\d+).*seed(\d+)', str(path))
        df[['template','per_map_size', 'map_size', 'seed']] = re.findall(r'-([a-zA-Z_]*)-n(\d+)in(\d+).*seed(\d+)', str(path))
        df[['per_map_size', 'map_size', 'seed']] = df[['per_map_size', 'map_size', 'seed']].apply(lambda x: int(x))

#         display(df)
        dfs.append(df)
    df = pd.concat(dfs, ignore_index=True)
    df.loc[df['template']=='mulneg', 'template']=''
    ordered_templates = ['', 'beginning', 'foo', 'pro_con', 'combined', 'all']
#     df = df.sort_values(by=['template'], key=lambda x: x.map({k:v for k,v in zip(ordered_templates, list(range(len(ordered_templates))))}))
#     display(df)
    df = pd.pivot_table(df, index=['per_map_size', 'map_size','template'], values=['p1', 'p5', 'mrr'],
                                aggfunc=[np.mean, np.std, np.count_nonzero], dropna=False)
    df = df[df['count_nonzero']['p1']==5]
    # display(df)

    formatter = lambda x: (x[0] if type(x[0])==str else format_f(x[0])+(f'\u00B1{format_f(x[1])}' if not pd.isna(x[1]) else '').replace('%',''))
    f_df = pd.DataFrame({col: zip(df['mean'][col], df['std'][col]) for col in df['mean'].columns})
    f_df.index = df.index
    f_df = f_df.applymap(formatter)
    
    f_df = f_df[['p1', 'p5', 'mrr']]
#     f_df.reset_index(level=0, inplace=True)
#     f_df.columns = f_df.columns.droplevel(2)

    f_df = f_df.reindex(ordered_templates, level=2)
    
    return f_df

## with samples results

In [ ]:
# results_df = pd.concat([read_results('avg'), read_results('annotated_samples_metrics')], keys=['test', 'samples'], axis=1)
# results_df.columns = results_df.columns.droplevel(0)
# results_df.reset_index(level=0)

## without samples results 

In [ ]:
results_df = read_results('avg')

In [ ]:
summary_df = results_df.query("template in ['', 'beginning', 'all']")
summary_df

In [ ]:
print(summary_df.to_latex())